In [ ]:
import pandas as pd
import numpy as np

# Загрузка данных
df = pd.read_csv('LizaAlert.csv')

# Проверка наличия необходимых столбцов
required_columns = ['status', 'gender']
if not all(col in df.columns for col in required_columns):
    missing = [col for col in required_columns if col not in df.columns]
    print(f"Ошибка: отсутствуют столбцы {missing}")
else:
    # Приводим статус к строке и очищаем от пробелов
    df['status'] = df['status'].astype(str).str.strip()
    df['gender'] = df['gender'].astype(str).str.strip().str.lower()
    
    # Создаем условия для замены
    is_empty = (df['status'] == '') | (df['status'] == 'nan') | (df['status'] == 'стоп') | (df['status'] == 'Cтоп')
    starts_with_prop = df['status'].str.lower().str.startswith('проп')
    starts_with_dead = df['status'].str.lower().str.startswith(('погиб', '(погиб)'))
    starts_with_alive = df['status'].str.lower().str.startswith('жив')
    
    # Создаем временные столбцы для замены
    conditions = [
        (is_empty | starts_with_prop) & (df['gender'] == 'муж'),
        (is_empty | starts_with_prop) & (df['gender'] == 'жен'),
        starts_with_dead & (df['gender'] == 'муж'),
        starts_with_dead & (df['gender'] == 'жен'),
        starts_with_alive & (df['gender'] == 'муж'),
        starts_with_alive & (df['gender'] == 'жен')
    ]
    
    choices = [
        'пропал',    # для мужчин (проп/пусто)
        'пропала',   # для женщин (проп/пусто)
        'погиб',     # для мужчин (погиб)
        'погибла',   # для женщин (погиб)
        'жив',       # для мужчин (жив)
        'жива'       # для женщин (жив)
    ]
    
    # Применяем замену
    df['status'] = np.select(conditions, choices, default=df['status'])
    
    # Обработка других значений пола
    other_gender_mask = ~df['gender'].isin(['муж', 'жен'])
    df.loc[other_gender_mask & (is_empty | starts_with_prop), 'status'] = 'пропал(а)'
    df.loc[other_gender_mask & starts_with_dead, 'status'] = 'погиб(ла)'
    df.loc[other_gender_mask & starts_with_alive, 'status'] = 'жив(а)'
    
    # Сохранение результата
    df.to_csv('updated_file.csv', index=False)
    S
    print("Обработка завершена успешно!")
    print("\nСтатистика замен:")
    print(df['status'].value_counts(dropna=False))

Обработка завершена успешно!

Статистика замен:
status
пропал(а)    28860
жив(а)        2767
пропал        2535
жив           1879
пропала       1215
жива           870
погиб(ла)      745
погиб          700
Стоп           695
погибла        182
Name: count, dtype: int64
